# Markov Decision Processes

## The "Karen" dilemma

Suppose today is your birthday. Someone has prepared a nice present and put it on a table across the room.

- Unfortunately some floor tiles are slippery.
- Some tiles are uneven, making it especially difficult to move into certain directions.
- And if that wouldn't be enough, very close to the table there is aunt "Karen" who keeps talking about politics all day.
- Did I mention that the room is entirely dark? You see no more than the tile you are currently standing on.

Obviously, you don't want to enter the room yourself and risk ruining your own birthday with repeated Karen-interactions.

A friend of yours offers to jump in and grab the present for you. You describe to them how they should navigate (**policy**) in the room. That description might not be perfect, since you don't know where the slippery tiles are. But - hey - your friend will report back how bad it really was.


The following sketch illustrates the situation in the room:

![](res/gridworld3.png)


## Task 1: Model the gridworld as a Markov Decision Process

- There are **nine states** and **at most** four actions in every state.
    ```
    states = (1, 2, 3, 4, 5, 6, 7, 8, 9)
    actions = {
        1: ('r', 'd'),
        2: ('r', 'd', 'l'),
        3: ('l', 'd'),
        4: ('u', 'r', 'd'),
        5: ('u', 'r', 'd', 'l'),
        6: ('u', 'd', 'l'),
        7: ('u', 'r'),
        8: ('u', 'r', 'l'),
        9: ('u', 'l')
    }
    ```

- The start state is `1`.
- The end state is `9`.
- Red numbers give the probability of ending up in the neighbor state (as indicated by the big arrow) if executing the specified action (little red arrow).
  e.g. If we are in state `8` and execute action `r`
  - There is a 80% chance to end up in state `7`. The remaining probability mass is distributed evenly among the other neighboring states (10% to end up in `5`, 10% to end up in `9`).
  - If we execute any action other than `r`, the state transition is deterministic: 100% chance of ending up in `7` if action is `l`; 100% chance of ending up in `5` if action is `u`.
- Any action in any state yields a reward of `-1`, because movement is exhausting.
   - Entering state `7` yields an additional reward of -10 (total: -11)
  - Entering state `9` yields an additional reward of +50 (total: 49)



In [66]:
class MDP(object):
    def __init__(self):
        pass

    def getStartState(self):
        """
        Return the start state of the MDP.
        """
        pass

    def isEndState(self, state):
        """
        Return True if 'state' is a terminal state.
        """
        pass

    def getActions(self, state):
        """
        Return a tuple of actions available in state `state`
        """
        pass

    def getSuccessorProbReward(self, state, action):
        """
        Helper function that returns both T(s, a, s') and R(s, a, s').
        Given a state 'state' and an action 'action', return a list of tuples (newState, transition probability, reward).
        I.e. Return a list of all possible successor states together with the chance of ending up in each successor and the reward received when doing so.
        """
        spr = {
            1: {'r': [(2, 1.0, -1)], 'd': [(4, 1.0, -1)]},
            2: {'r': [(3, 1.0, -1)], 'd': [(5, 1.0, -1)], 'l': [(1, 1.0, -1)]},
            3: {'d': [(6, 0.01, -1), (2, 0.99, -1)], 'l': [(2, 1.0, -1)]},
            4: {'u': [(1, 0.3, -1),(5, 0.3, -1),(7, 0.4, -11)], 'r': [(1, 0.3, -1),(5, 0.3, -1),(7, 0.4, -11)], 'd': [(1, 0.3, -1),(5, 0.3, -1),(7, 0.4, -11)]},
            5: {'u': [(2, 1.0, -1)], 'r': [(8, 0.7, -1), (6, 0.1, -1), (4, 0.1, -1), (2, 0.1, -1)], 'd': [(8, 1.0, -1)], 'l': [(4, 1.0, -1)]},
            # TODO

        }
        if state in spr:
            if action in spr[state]:
                return spr[state][action]
        return []

    def getDiscount(self):
        """
        How much to value future rewards compared to the immediate reward.
        1 means: All rewards contribute to the value of a state equally - no matter how far in the future they might occur.
        0.5 means: R0 + 0.5 * R1 + 0.5 * 0.5 * R2 + ...  The value of a state is controlled 50% by the immediate reward R0 and 50% by all future rewards R1, R2,....
        """
        return 1.0

    def getStates(self):
        states = (
              1, 2, 3,
              4, 5, 6,
              7, 8, 9
        )
        return states

In [67]:
mdp = MDP()
# TODO: Call methods to check if they return what you expect

## Task 2: Policy Evaluation

Given a policy, what is the sum of the (discounted) rewards when following that policy?

The sum of the (discounted) rewards is called the **Return**.

Each time we execute the policy, we end up with a different sequence of states and thus a different sequence of rewards (return).

The **Value** of a policy is the *average* **Return**s we get when following that policy.


The policy we want to try looks as follows:
```
policy = {1: 'd', 2:'d', 3:'d', 4:'r', 5:'r', 6:'d', 7:'u', 8:'r', 9:'u'}
```
Given some state, the policy prescribes which action to take.
Now, use the **policy evaluation** algorithm to compute how good this policy is (on average).

- The `immediateReward` is the reward we receive when we enter a state
- The $T(s, a, s')$ is the transition probability: The probability of ending up in state $s'$ when we take action $a$ in state $s$.
- `gamma` is the discount factor telling us the future total reward should be weighted compared to the immediate reward.

A sketch of the algorithm:
```
V = {s: 0 for s in states}
for i in range(n_iterations):
    for s in states:
        newV = 0.0
        a = policy[s]
        for s' in successorsOfS:
            newV += T(s, a, s') * (immediateReward + gamma * V[s'])
        V[s] = newV
```
**Note:** The value of a terminal state is always 0.

In [71]:
N = 100000                             # Max iterations
eps = 1e-100                          # Error tolerance: epsilon
states = mdp.getStates()              # List of states
V = {s: 0 for s in states}            # Initialize value function
policy = {1: 'd', 2:'d', 3:'d', 4:'r', 5:'r', 6:'d', 7:'u', 8:'r', 9:'u'} # A given policy

In [72]:
for i in range(N):
    for s in states:
        newV = 0.0
        if mdp.isEndState(s):
            newV = 0.0
        else:
            a = policy[s]
            for sn in mdp.getSuccessorProbReward(s, a):
                newV += sn[1] * (sn[2] + mdp.getDiscount() * V[sn[0]])
        V[s] = newV

In [73]:
V

{1: -9.5,
 2: -3.1666666666666665,
 3: -4.134999999999999,
 4: -8.5,
 5: -2.1666666666666665,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0}

The value of a policy is the average value over all states:

In [63]:
sum(V.values()) / len(V)

-58.10065359477111

In [64]:
V

{1: -91.1764705882351,
 2: -72.17647058823513,
 3: -71.96470588235277,
 4: -90.1764705882351,
 5: -71.17647058823513,
 6: 49.0,
 7: -91.1764705882351,
 8: -84.0588235294116,
 9: 0.0}

## Task3: Value Iteration

So far, a given policy told us which action to take in each state.
The value of each state was then a consequence of following that policy.

Now, let's calculate the optimal policy for this MDP.

All we need to do is to insert another loop that runs over all available actions in any state.
The action with the highest value is the one we store in the optimal policy (`policy`).

In [65]:
N = 10000                             # Max iterations
eps = 1e-100                          # Error tolerance epsilon
states = mdp.getStates()              # List of states
V = {s: 0 for s in states}            # Initialize value function
policy = {s: None for s in states}    # Initialize policy

In [260]:
import math

for i in range(N):
    max_diff = 0.0                  # Initialize max difference
    V_new = {s: 0 for s in states}  # Initialize temporary values

    for s in mdp.getStates():

        # TODO

        pass

In [261]:
V

{1: 22.59999999999996,
 2: 23.59999999999996,
 3: 22.85399999999996,
 4: 18.199999999999957,
 5: 24.59999999999996,
 6: 49.0,
 7: 22.59999999999996,
 8: 23.59999999999996,
 9: 0}

In [262]:
policy

{1: 'r', 2: 'd', 3: 'd', 4: 'u', 5: 'r', 6: 'd', 7: 'r', 8: 'u', 9: None}